In [16]:
import pandas as pd


In [17]:
file = '/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/Jan19.txt'
out='/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/criteria_above_discard_redundant_clinsig.txt'

In [19]:
with open(file,'r') as f:
    with open(out,'w') as f_out:
        line=f.readline()
        f_out.write('clinvar_id;review_status;clinical_sig;uniprot_kb;variant_type;hgvs_p;missense\n')
        while True:
            line=f.readline()

            if line.count(';')>6:
                discard_num=line.count(';')-6+3
                line_new=';'.join(line.split(';')[:3]+line.split(';')[discard_num:])
                print(set(line.split(';'))-set(line_new.split(';')),' is discarded')
                f_out.write(line_new)
            else:
                f_out.write(line)
            if not line:break

{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' association', ' risk factor'}  is discarded
{' association'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' risk factor'}  is discarded
{' association'}  is discarded
{' risk factor', ' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is discarded
{' other'}  is 

# Inspect data

In [20]:
data=pd.read_csv(out,delimiter=';')
data.head()

,clinvar_id,review_status,clinical_sig,uniprot_kb,variant_type,hgvs_p,missense
0,SCV002232601,"criteria provided, single submitter",Pathogenic,P04181#VAR_000578,Variation,P04181:p.Arg271Lys,missense variant
1,SCV002291827,"criteria provided, single submitter",Uncertain significance,P04181#VAR_015648,Variation,P04181:p.Gln90Glu,missense variant
2,SCV001445903,"criteria provided, single submitter",Likely pathogenic,P12955#VAR_011615,Variation,P12955:p.Gly278Asp,missense variant
3,SCV001449002,"criteria provided, single submitter",Pathogenic/Likely pathogenic,P12955#VAR_004405,Variation,P12955:p.Gly448Arg,missense variant
4,SCV000935565,"criteria provided, single submitter",Likely pathogenic,P21359#VAR_002663,Variation,P21359:p.Leu1953Pro,missense variant


In [22]:
data.clinical_sig.unique()

array(['Pathogenic', 'Uncertain significance', 'Likely pathogenic',
       'Pathogenic/Likely pathogenic', 'Benign', 'Likely benign',
       'Conflicting interpretations of pathogenicity',
       'Benign/Likely benign', 'drug response', 'other',
       'Likely risk allele', 'risk factor',
       'Pathogenic/Likely pathogenic/Established risk allele',
       'Established risk allele', 'Pathogenic/Pathogenic, low penetrance',
       'association'], dtype=object)

In [34]:
data.variant_type.unique()

array(['Variation', 'single nucleotide variant', 'copy number gain'],
      dtype=object)

# Process clinical significance

['Pathogenic', 'Uncertain significance', 'Likely pathogenic',
       'Pathogenic/Likely pathogenic', 'Benign', 'Likely benign',
       'Conflicting interpretations of pathogenicity',
       'Benign/Likely benign', 'drug response', 'other',
       'Likely risk allele', 'risk factor',
       'Pathogenic/Likely pathogenic/Established risk allele',
       'Established risk allele', 'Pathogenic/Pathogenic, low penetrance',
       'association']

In [29]:
def if_positive_or_negative(string_list):
    label=[]
    for string in string_list:
        if string in ['Pathogenic', 
                    'Pathogenic/Likely pathogenic', 
                    'probable-pathogenic',
                    'Likely pathogenic', 
                    'pathologic', 
                    'pathogenic',
                    'likely pathogenic',
                    'Pathogenic/Likely pathogenic/Established risk allele',
                    'likely pathogenic - adrenal pheochromocytoma',
                    'Pathogenic/Pathogenic, low penetrance',
                    'Pathogenic, low penetrance']:
            label.append(1)
        elif string in ['Benign',
                    'Likely benign',
                    'Likely Benign',
                    'Benign/Likely benign',
                    'non-pathogenic', 
                    'benign', 'probable-non-pathogenic', 'Likely Benign', 'probably not pathogenic',
        ]:
            label.append(-1)

        else: label.append(0)
    return label

In [30]:
data['label']=if_positive_or_negative(data['clinical_sig'])

## Generate Positive Clinvar File and Negative Clinvar File

In [32]:
data[data['label']==1].to_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/clinvar_pos.csv',sep=';')
data[data['label']==-1].to_csv('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/clinvar_neg.csv',sep=';')

In [35]:
data[data['label']==-1]

,clinvar_id,review_status,clinical_sig,uniprot_kb,variant_type,hgvs_p,missense,label
19,SCV000361195,"criteria provided, single submitter",Benign,P0C7Q2#VAR_044331,Variation,P0C7Q2:p.Ala69Ser,missense variant,-1
21,SCV001325434,"criteria provided, single submitter",Likely benign,O60393#VAR_036636,Variation,O60393:p.Arg355His,missense variant,-1
55,SCV000451797,"criteria provided, single submitter",Likely benign,Q9NRM0#VAR_065775,Variation,Q9NRM0:p.Arg198Cys,missense variant,-1
58,SCV001136513,"criteria provided, single submitter",Benign,Q9NP91#VAR_052068,Variation,Q9NP91:p.Thr199Met,missense variant,-1
79,SCV001138159,"criteria provided, single submitter",Benign,P15169#VAR_042415,Variation,P15169:p.Gly178Asp,missense variant,-1
...,...,...,...,...,...,...,...,...
32189,SCV002738545,"criteria provided, single submitter",Benign,P17813#VAR_014764,Variation,P17813:p.Asp366His,missense variant,-1
32191,SCV002612763,"criteria provided, single submitter",Benign,Q14896#VAR_074545,Variation,Q14896:p.Arg1138His,missense variant,-1
32192,SCV002613575,"criteria provided, single submitter",Likely benign,Q14896#VAR_020571,Variation,Q14896:p.Leu383Val,missense variant,-1
32196,SCV002614959,"criteria provided, single submitter",Benign,Q8TAB3#VAR_067489,Variation,Q8TAB3:p.Asn1134His,missense variant,-1


In [12]:
def generate_example(start,end):
    with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/ClinVarFullRelease_00-latest.xml', 'r') as f_in:
        with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/example_'+str(start)+'_'+str(end)+'.xml','w') as f_out:
            line_count=0
            while True:
                line=f_in.readline()
                line_count+=1
                if line_count>start and line_count<end:
                    f_out.write(line)
                if line_count>end:
                    break

def find_line(content):
    with open('/home/grads/z/zshuying/Documents/shuying/ppi_mutation/data/clinvar/ClinVarFullRelease_00-latest.xml', 'r') as f_in:
            line_count=0

            while True:
                line=f_in.readline()
                line_count+=1

                if content in line:
                    print(line_count)
                    break

In [41]:
generate_example(16932274,16934274)

In [40]:
find_line('SCV000345973')

16933274


In [ ]:
RCV000203850 L936 (932-1335)
SCV000259045 L1117

delayed one

SCV000258941 L1257